# enVector API Flow Example

This example shows the end-to-end workflow of **enVector** Python SDK (`es2`).

In this notebook, we'll show the following works step-by-step:
1. how vector encryption works
2. how metadata encryption works
3. how encrypted similarity search works
4. how decryption works

by running the APIs explicitly. 
This example ensures that the server sees only the encrypted data while the raw data remain unexposed.



## Import SDK

First, you should install and import the `es2` package to use enVector Python APIs.
Before installing, make sure you have Python 3.12 and a virtual environment on your system. 

For more details, see [SDK installation](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh/get-started/installation/client-sdk) section in [enVector Documents](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh).

In [ ]:
import es2

### Connect to the enVector endpoint server

First, we need to connect to the enVector endpoint server. The `init_connect()` is included in `init()`.

For more details, see [Connection Configuration](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh/user-guide/initialize/connection-configuration) section in [enVector Documents](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh).

In [ ]:
# Connect to the enVector end-point server with address and access token (if required)
es2.init_connect(
    address="localhost:50050",
    # access_token="...",  # if needed
)

# Or you can call es2.init() for connection and configuration without key registration
# es2.init(address="0.0.0.0:50050", key_path=key_path, auto_key_setup=False)

## Key Control

### Generate FHE Keys

To use enVector, we should generate a key, which is the basis of the ciphertext for fully homomorphic encryption (FHE).

By key generation, all keys will be saved to `key_path` identified by `key_id` and generated as:
- `SecKey.bin` for the Secret key,
- `EncKey.bin` for the Encryption key,
- `EvalKey.bin` for the Evaluation key.

In the case that metadata is sensitive, you can set `metadata_encryption` as `True` to generate the metadata encryption key:
- `MetadataKey.bin` for the Metadata Encryption key.

`eval_mode` can be set as `rmp` (the default) or `mm` (advanced). `rmp` mode is the default key setting, and `mm` provides better performance for large-scale datasets, but it currently doesn't support the query encryption.

For a more detailed description, see [Key Generation](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh/user-guide/advanced-user-guide/key-generation) in [enVector Documents](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh).

In [ ]:
from es2.crypto import KeyGenerator

# Generate keys locally under key_path = keys/crypto-qs
keygen = KeyGenerator(
    key_path="keys/crypto-qs", 
    eval_mode="rmp",
    metadata_encryption=True,
)
keygen.generate_keys()

### Index Configuration
Now we can configure the index config with the key ID. 
The index config includes query encryption, index encryption, metadata encryption, and so on.

For more details, see [Index Config](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh/user-guide/initialize/index-configuration/4.-indexconfig) in [enVector Documents](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh).

In [ ]:
# Configure index and key config
es2.init_index_config(
    key_path="keys",
    key_id="crypto-qs",
    query_encryption="plain",
    auto_key_setup=False,
    metadata_encryption=True,
)

### Register Key

We should register and load the key with the enVector server. The server cannot encrypt or decrypt, so we send the evaluation key only for secure operations.

For more details, see [Implementation Steps](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh/user-guide/connection/key-management#implementation-steps) in [enVector Documents](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh).

In [ ]:
# Register and load the key on the server
es2.register_key(key_id="crypto-qs")
es2.load_key(key_id="crypto-qs")

If we want to know specific information about the registered key, we can use the `get_key_list()` and `get_key_info()` methods.

In [ ]:
es2.get_key_list()

In [ ]:
es2.get_key_info("crypto-qs")

## Prepare Index

### Create Index

Before inserting data, we first create an `Index` with the dimension of the vectors you want to use.

For more details, see [Creation](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh/user-guide/encrypted-index/creation) in [enVector Documents](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh).

In [ ]:
# Create an index bound to this key config
index = es2.create_index("crypto_qs_index", dim=512)

In [ ]:
index.index_config

### Prepare data

Here, we prepare sample data for inserting into the enVector server. Note that the vectors must be L2-normalized before insertion.

In [ ]:
import numpy as np

# Generate sample normalized vectors and metadata
vecs = np.random.rand(10, 512)
vecs = vecs / np.linalg.norm(vecs, axis=1, keepdims=True)
metadata = [f"Item {i+1}" for i in range(10)]

### Encrypt Items

To securely store or search vectors in the enVector system, we need to encrypt them using the generated encryption keys.

The `Cipher` class provides a unified interface for both encryption and decryption of vectors and scores. You can provide the encryption and decryption key paths when initializing, or you can specify them directly when calling the `encrypt` or `decrypt` methods. This flexibility allows you to manage keys according to your workflow and security requirements.

For example, you can initialize a `Cipher` with both key paths for convenience, or you can create a `Cipher` with only the dimension and provide the key paths at each operation. This makes it easy to use the same `Cipher` instance in different contexts or with different keys as needed.

In the case of metadata encryption, you can use `encrypt_metadata` with the generated metadata encryption key `MetadataKey.bin`.

For more details, see [Encryption](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh/user-guide/advanced-user-guide/cipher/encryption) in [enVector Documents](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh).



In [ ]:
from es2.crypto import Cipher
from es2.utils.aes import encrypt_metadata

enc_key_path="./keys/crypto-qs/EncKey.bin"
metadata_key_path="./keys/crypto-qs/MetadataKey.bin"

cipher = Cipher(dim=512, enc_key_path=enc_key_path)

# Explicitly encrypt each item (vector) before sending
enc_vectors = [cipher.encrypt(v, "item") for v in vecs]
print("Encrypted Vectors\n", enc_vectors[0])

# Explicitly encrypt metadata
enc_metadata = [encrypt_metadata(item, key_path=metadata_key_path) for item in metadata]
print("Encrypted Metadata\n", enc_metadata[0])

### Insert the Encrypted Vectors

After encrypting all vectors, we're ready to insert the encrypted vectors into the created index on the enVector server.
This sends a list of encrypted vectors and the corresponding encrypted metadata to the enVector server and stores them in the specified index.

For more details, see [Insert](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh/user-guide/insert) in [enVector Documents](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh).

In [ ]:
# Insert encrypted items with metadata
index.insert(enc_vectors, enc_metadata)

## Encrypted Similarity Search

### Prepare query

First, prepare a plaintext query for encrypted similarity search.

If you want to encrypt the query for more secure similarity search, see `03-simple-rag.ipynb` for more details.

In [ ]:
query = vecs[0]

### Encrypted search on the index

Now we're ready to perform an encrypted similarity search. 
Once the encrypted vector index and query vectors are ready, we can perform the encrypted search on encrypted data without decrypting it.
In this example, we'll show how the `Index.search` works, running the underlying APIs step-by-step explicitly.
The `scoring` API performs the encrypted similarity search, resulting in ciphertexts of similarity scores. 

For more details, see [Scoring](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh/user-guide/search/scoring) in the [enVector Documents](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh).

In [ ]:
search_index = es2.Index("crypto_qs_index")

# Server performs encrypted similarity scoring; response is encrypted
encrypted_scores = search_index.scoring(query)[0]
encrypted_scores

### Decrypt search results

The result of the encrypted search is ciphertext, which means that the server could not see the original data. To see the result, we need to decrypt the outputs with the client's secret key. The encrypted search result includes similarity scores for each vector, which become available after decryption. 

For more details, see the [Decrypting Scores](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh/user-guide/search/decrypting-scores) in the [enVector Documents](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh).

In [ ]:
scores = cipher.decrypt_score(encrypted_scores, sec_key_path="keys/crypto-qs/SecKey.bin")
scores

### Extract Top-k Relevance

To extract relevant vectors, we first identify the top-k indices from the decrypted result vectors. These indices correspond to the most similar vectors in the encrypted index.

For more details, see the [Retrieving Top-k Metadata](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh/user-guide/search/retrieving-top-k-metadata) in the [enVector Documents](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh).

In [ ]:
topk = search_index.get_topk_metadata_results(scores, top_k=3, output_fields=["metadata"])
print(*topk, sep="\n")

### Decrypt Metadata

The resulting metadata is encrypted, as we inserted the encrypted ones for security.
Therefore, we decrypt the retrieved metadata to see the original data.

In [ ]:
from es2.utils.aes import decrypt_metadata

retrievals = [decrypt_metadata(item["metadata"], key_path=metadata_key_path) for item in topk]
print(*retrievals, sep="\n")

### Clean up

We can delete the created index and the registered key when they are no longer needed.

In [ ]:
es2.drop_index("crypto_qs_index")

In [ ]:
es2.delete_key("crypto-qs")

For more detailed API reference, see [enVector Documents](https://cryptolab.gitbook.io/envector/rU12grf1Q12UCpb0sxoh).